In [5]:
##########################################
############Sentiment Analysis############
##########################################

############# Data Collection ############

from bs4 import BeautifulSoup
import re
import sys
import os
#import urllib.request 
import urllib.request, urllib.parse, urllib.error

#import search

%run search.ipynb
#%run config.ipynb

#from search import search 
outputfilename = "lyrics.csv"
client_access_token = "AZlQ_g-6H3cOH5vQzCR_uBkceszUpTxL7Jl_ISA6BkQbeN5-W5peit9krGty_uBy"

artists = ["cigarettes after sex", "eric clapton", "damien rice", "dire straits", \
           "the black keys", "Eminem", "Porcupine tree", "Northlane", \
           "incubus", "radiohead"]

def get_lyrics(url):
    request = urllib2.Request(url)
    request.add_header("Authorization", "Bearer " + client_access_token)
    request.add_header("User-Agent",
                       "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)")
    page = urllib2.urlopen(request)
    soup = BeautifulSoup(page, "lxml")
    lyrics = soup.find("div", class_= "lyrics")
    return lyrics.text

for artist in artists:
    a = search(artist, outputfilename, client_access_token)
    
    #urls = list(map(lambda t: t[3], a))
    urls = [t[0] for t in a]
    f = open('lyrics/' + artist, 'wb')
    for url in urls:
        lyrics = get_lyrics(url)
        f.write(lyrics.encode("utf8"))
    f.close()
    

URLError: <urlopen error _ssl.c:761: The handshake operation timed out>

page 1; num hits 10
page 2; num hits 10
page 3; num hits 10
page 4; num hits 10
page 5; num hits 10
page 6; num hits 10
page 7; num hits 10
page 8; num hits 10
page 9; num hits 10
page 10; num hits 10
page 11; num hits 10
page 12; num hits 10
page 13; num hits 10
page 14; num hits 10
page 15; num hits 10
page 16; num hits 10
page 17; num hits 5
page 18; num hits 9
page 19; num hits 10
page 20; num hits 10
page 21; num hits 10
page 22; num hits 10
page 23; num hits 10
page 24; num hits 9
page 25; num hits 9
page 26; num hits 10
page 27; num hits 10
page 28; num hits 10
page 29; num hits 10
page 30; num hits 10
page 31; num hits 10
Timeout raised and caught
page 32; num hits 10
page 33; num hits 9
page 34; num hits 10
page 35; num hits 9
page 36; num hits 10
Timeout raised and caught
page 37; num hits 10
page 38; num hits 10
page 39; num hits 10
page 40; num hits 8
page 41; num hits 10
page 42; num hits 10
page 43; num hits 9
page 44; num hits 10
page 45; num hits 10
page 46; num hits 10

TypeError: 'NoneType' object is not iterable

In [6]:
########## Data Cleaning #########

for artist in artists:
    f = open('lyrics/' + artist, 'rb')
    all_words = ''
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        all_words += this_sentence
    f.close()
    
    all_words = re.sub(r'[\(\[].*?[\)\]]', '', all_words)
    
    all_words = os.linesep.join([s for s in all_words.splitlines() if s])
    
    f = open('lyrics/' + artist + '-cleaned', 'wb')
    f.write(all_words.encode('utf-8'))
    f.close()

FileNotFoundError: [Errno 2] No such file or directory: 'lyrics/cigarettes after sex'

In [7]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (8,6)

df = pd.DataFrame(columns = ('artist', 'words'))
i = 0
for artist in artists:
    f = open('lyrics/' + artist + '-cleaned', 'rb')
    num_words = 0
    all_text = ''
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        num_words_this = len(this_sentence.split(" "))
        num_words += num_words_this
        
    df.loc[i] = (artist, num_words)
    i += 1
df.plot.bar(x = 'artist', y='words', title = 'Number of Words for each Artist');
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'lyrics/cigarettes after sex-cleaned'

In [ ]:
from nltk.corpus import stopwords

df = pd.DataFrame(columns=('artist', 'lexicalrichness'))
for artist in artists:
    f = open('lyrics/' + artist + '-cleaned', 'rb')
    all_words = ''
    num_words = 0
    raw_text = ""
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        raw_text += this_sentence
            
    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    
    a = len(set(filtered_words))
    b = len(words)
    df.loc[i] = (artist, (a/float(b))*100)
    i+=1
    
df.plot.bar(x='artist', y='lexicalrichness', title='Lexical richness of each Artist');
plt.show()    

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0
for artist in artists:
    num_positive = 0
    num_negative = 0
    num_neutral = 0

    f = open('lyrics/' + artist + '-cleaned', 'rb')
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        comp = sid.polarity_scores(this_sentence)
        comp = comp['compound']
        if comp >= 0.5:
            num_positive += 1
        elif comp > -0.5 and comp < 0.5:
            num_neutral += 1
        else:
            num_negative += 1

    num_total = num_negative + num_neutral + num_positive
    percent_negative = (num_negative/float(num_total))*100
    percent_neutral = (num_neutral/float(num_total))*100
    percent_positive = (num_positive/float(num_total))*100
    df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
    i+=1
                 
df.plot.bar(x='artist', stacked=True)
plt.show()          